### Prapengolahan

In [1]:
import pandas as pd
import numpy as np
import os
import olah_data as od
pd.options.display.max_columns = None

In [2]:
# DEFINE DATA MENTAH

kategori = 'call center'
# kategori_call_center = 'regular banking'
kategori_call_center = 'car insurance'
# kategori = 'twitter'
# kategori = 'email'
# kategori = 'online chat'

if kategori=='call center':
    file_data_mentah = 'data mentah/data mentah '+kategori+' - '+kategori_call_center+' - 2021.xlsx'
else:
    file_data_mentah = 'data mentah/data mentah '+kategori+' - 2021.xlsx'
file_bobot = "bobot/bobot perhitungan "+kategori+'.xlsx'

data_mentah = pd.read_excel(file_data_mentah)

# LOAD DATA BOBOT
daftar_bobot = pd.read_excel(file_bobot, sheet_name='indeks')
data_bobot = od.data_bobot(daftar_bobot)

In [3]:
# display(data_mentah.head(3))
# display(data_bobot.head(3))

In [4]:
# GANTI NAMA BULAN KE NOMOR BULAN
data_mentah.Bulan = od.ganti_bulan(data_mentah.Bulan,'nama bulan','nomor bulan')

# DROP NULL ROW
data_mentah = od.drop_null_row(data_mentah)

# DEFINE LIST BRAND
list_brand = data_mentah['Brand'].unique()
if kategori=='call center':
    brand_ivr, brand_nonivr = od.get_ivr_nonivr(data_mentah, list_brand)

# TABEL CROSSTAB
crosstab_rata_transpose, crosstab_rata= od.crosstab_rata_transpose(data_mentah, list_brand, data_bobot)

In [5]:
print("brand ivr")
print(brand_ivr)
print("")
print("brand nonivr")
print(brand_nonivr)

brand ivr
[]

brand nonivr
['24 Hour Customer Care Asuransi Sinar Mas', 'ACA Hotline', 'BCA Insurance', 'Call Center Jasindo', 'Call TIA', 'Dering Adira', 'Garda Akses Asuransi Astra']


In [6]:
crosstab_rata.head()

DIMENSI                                                                                     Access  \
KPI                                                                                  Accessibility   
SUB KPI                                                                              Accessibility   
ASPEK                                            Banyaknya mencoba hingga mendapatkan nada sambung   
KODE                                                                            ncoba_nadaSambung1   
Bulan   Brand                                                                                        
Januari 24 Hour Customer Care Asuransi Sinar Mas                                              87.5   
        ACA Hotline                                                                           87.5   
        BCA Insurance                                                                         87.5   
        Call Center Jasindo                                                                   87.5   
        Call TIA                                                                              87.5   

DIMENSI                                                              \
KPI                                                                   
SUB KPI                                                               
ASPEK                                                                 
KODE                                             ncoba_nadaSambung2   
Bulan   Brand                                                         
Januari 24 Hour Customer Care Asuransi Sinar Mas               12.5   
        ACA Hotline                                            12.5   
        BCA Insurance                                          12.5   
        Call Center Jasindo                                    12.5   
        Call TIA                                               12.5   

DIMENSI                                                              \
KPI                                                                   
SUB KPI                                                               
ASPEK                                                                 
KODE                                             ncoba_nadaSambung3   
Bulan   Brand                                                         
Januari 24 Hour Customer Care Asuransi Sinar Mas                0.0   
        ACA Hotline                                             0.0   
        BCA Insurance                                           0.0   
        Call Center Jasindo                                     0.0   
        Call TIA                                                0.0   

DIMENSI                                                              \
KPI                                                                   
SUB KPI                                                               
ASPEK                                                                 
KODE                                             ncoba_nadaSambung4   
Bulan   Brand                                                         
Januari 24 Hour Customer Care Asuransi Sinar Mas                0.0   
        ACA Hotline                                             0.0   
        BCA Insurance                                           0.0   
        Call Center Jasindo                                     0.0   
        Call TIA                                                0.0   

DIMENSI                                                              \
KPI                                                                   
SUB KPI                                                               
ASPEK                                                                 
KODE                                             ncoba_nadaSambung5   
Bulan   Brand                                                         
Januari 24 Hour Customer Care Asuransi Sinar Mas                0.0   
        ACA Hotline               

### perhitungan indeks

In [7]:
# DEFINE LIST BULAN
list_bulan= pd.Series(data_mentah.Bulan.unique())
list_bulan = list(od.ganti_bulan(list_bulan,'nomor bulan','nama bulan'))


if kategori=='call center':
    nilai_subaspek = od.nilai_sub_aspek(kategori, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose,
                                        brand_ivr, brand_nonivr)
    nilai_aspek = od.nilai_aspek_with_sub(kategori, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, nilai_subaspek,
                                          brand_ivr, brand_nonivr)    
elif kategori=='email':
    nilai_aspek = od.nilai_aspek_with_sub(kategori, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, None)
else:
    nilai_subaspek = od.nilai_sub_aspek(kategori, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose)
    nilai_aspek = od.nilai_aspek_with_sub(kategori, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, nilai_subaspek)    
nilai_aspek_only = od.nilai_aspek_without_sub(kategori, daftar_bobot, list_bulan, list_brand, crosstab_rata_transpose, nilai_aspek)
df_aspek_pivot = od.df_aspek_pivot(nilai_aspek, nilai_aspek_only, list_bulan)
tabel_summary_aspek = od.tabel_summary(df_aspek_pivot, list_bulan)

In [8]:
df_aspek_pivot.iloc[1:4,:]

Bulan                                                                                                                                    Januari  \
Brand                                                                                                   24 Hour Customer Care Asuransi Sinar Mas   
Dimensi KPI              SUB KPI          Nomor Kode Aspek                                                                                         
Access  Availability     Availability     2     2    Banyaknya mencoba hingga dapat terhubung ke mes...                                     97.5   
        Connection Speed Connection Speed 3     3    Lamanya menunggu sampai dengan terdengarnya Sal...                                    100.0   
        Availability     Availability     4     4    Banyaknya mencoba hingga dapat terhubung ke Cus...                                    100.0   

Bulan                                                                                                                \
Brand                                                                                                   ACA Hotline   
Dimensi KPI              SUB KPI          Nomor Kode Aspek                                                            
Access  Availability     Availability     2     2    Banyaknya mencoba hingga dapat terhubung ke mes...        97.5   
        Connection Speed Connection Speed 3     3    Lamanya menunggu sampai dengan terdengarnya Sal...        95.0   
        Availability     Availability     4     4    Banyaknya mencoba hingga dapat terhubung ke Cus...       100.0   

Bulan                                                                                                                  \
Brand                                                                                                   BCA Insurance   
Dimensi KPI              SUB KPI          Nomor Kode Aspek                                                              
Access  Availability     Availability     2     2    Banyaknya mencoba hingga dapat terhubung ke mes...          97.5   
        Connection Speed Connection Speed 3     3    Lamanya menunggu sampai dengan terdengarnya Sal...         100.0   
        Availability     Availability     4     4    Banyaknya mencoba hingga dapat terhubung ke Cus...          97.5   

Bulan                                                                                                                        \
Brand                                                                                                   Call Center Jasindo   
Dimensi KPI              SUB KPI          Nomor Kode Aspek                                                                    
Access  Availability     Availability     2     2    Banyaknya mencoba hingga dapat terhubung ke mes...               100.0   
        Connection Speed Connection Speed 3     3    Lamanya menunggu sampai dengan terdengarnya Sal...               100.0   
        Availability     Availability     4     4    Banyaknya mencoba hingga dapat terhubung ke Cus...                97.5   

Bulan                                                                                                             \
Brand                                                                                                   Call TIA   
Dimensi KPI              SUB KPI          Nomor Kode Aspek                                                         
Access  Availability     Availability     2     2    Banyaknya mencoba hingga dapat terhubung ke mes...     97.5   
        Connection Speed Connection Speed 3     3    Lamanya menunggu sampai dengan terdengarnya Sal...    100.0   
        Availability     Availability     4     4    Banyaknya mencoba hingga dapat terhubung ke Cus...     90.0   

Bulan                                                                                                                 \
Brand                                                                                    

In [9]:
if kategori=='call center':
    df_subkpi_pivot = od.df_subkpi_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_aspek_pivot,
                                         brand_ivr, brand_nonivr)
    df_kpi_pivot = od.df_kpi_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_subkpi_pivot,
                                   brand_ivr, brand_nonivr)
    df_dimensi_pivot = od.df_dimensi_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_kpi_pivot,
                                           brand_ivr, brand_nonivr)
    df_ccsei_pivot = od.df_ccsei_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_dimensi_pivot,
                                       brand_ivr, brand_nonivr)    
else:
    df_subkpi_pivot = od.df_subkpi_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_aspek_pivot)
    df_kpi_pivot = od.df_kpi_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_subkpi_pivot)
    df_dimensi_pivot = od.df_dimensi_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_kpi_pivot)
    df_ccsei_pivot = od.df_ccsei_pivot(kategori, daftar_bobot, list_bulan, list_brand, df_dimensi_pivot)    
    
tabel_summary_subkpi = od.tabel_summary(df_subkpi_pivot, list_bulan)
tabel_summary_kpi = od.tabel_summary(df_kpi_pivot, list_bulan)
tabel_summary_dimensi = od.tabel_summary(df_dimensi_pivot, list_bulan)

In [10]:
df_ccsei_pivot

,Januari,Februari,Maret,~Average
Brand,,,,
24 Hour Customer Care Asuransi Sinar Mas,84.05,89.56,92.11,88.57
ACA Hotline,88.96,78.74,65.93,77.88
BCA Insurance,92.57,94.69,87.63,91.63
Call Center Jasindo,88.81,92.11,77.40,86.11
Call TIA,90.03,94.58,92.27,92.29
Dering Adira,83.40,92.32,87.68,87.80
Garda Akses Asuransi Astra,87.88,89.87,92.83,90.19
~Industri,87.96,90.27,85.12,87.78


In [11]:
# Load Data Bobot Engagement
bobot_engagement = pd.read_excel(file_bobot, sheet_name='engagement')
if kategori=='call center':
    tabel_be_avg, df_eng_final_pivot = od.engagement(kategori, bobot_engagement, df_aspek_pivot, list_bulan, list_brand, 
                                                     None,  brand_ivr, brand_nonivr)
elif kategori=='twitter' or kategori=='online chat':
    tabel_be_avg, df_eng_final_pivot = od.engagement(kategori, bobot_engagement, df_aspek_pivot, list_bulan, list_brand)    
else:
    tabel_be_avg, df_eng_final_pivot = od.engagement(kategori, bobot_engagement, df_aspek_pivot, list_bulan, list_brand, 
                                                     crosstab_rata_transpose)
    
# Load Data Bobot Impact perbaikan
bobot_perbaikan = pd.read_excel(file_bobot, sheet_name='perbaikan')
if kategori=='call center':
    area_perbaikan = od.area_perbaikan(kategori, data_mentah, bobot_perbaikan, list_bulan, list_brand,
                                       brand_ivr, brand_nonivr)
else:
    area_perbaikan = od.area_perbaikan(kategori, data_mentah, bobot_perbaikan, list_bulan, list_brand)    

In [12]:
area_perbaikan.head(3)

Frekuensi Perbaikan  \
NOMOR LEVEL 1 NOMOR LEVEL 2 KODE               SUB KPI       ASPEK LAPORAN                                     Brand                                                           
1.0           a             ncoba_nadaSambung1 Accessibility Banyaknya mencoba hingga mendapatkan nada sambung 24 Hour Customer Care Asuransi Sinar Mas                  1.0   
                                                                                                               ACA Hotline                                               1.0   
                                                                                                               BCA Insurance                                             3.0   

                                                                                                                                                         _Priority Index  
NOMOR LEVEL 1 NOMOR LEVEL 2 KODE               SUB KPI       ASPEK LAPORAN                                     Brand                                                      
1.0           a             ncoba_nadaSambung1 Accessibility Banyaknya mencoba hingga mendapatkan nada sambung 24 Hour Customer Care Asuransi Sinar Mas             0.02  
                                                                                                               ACA Hotline                                          0.02  
                                                                                                               BCA Insurance                                        0.06

### Simpan Data ke Ms Excel

In [13]:
if kategori=='call center':
    nama_output = "tabel output/tabel output "+kategori+" - "+kategori_call_center+".xlsx"
else:
    nama_output = "tabel output/tabel output "+kategori+".xlsx"

list_simpan = [data_mentah, crosstab_rata, crosstab_rata_transpose, tabel_summary_aspek, tabel_summary_subkpi, tabel_summary_kpi,
              tabel_summary_dimensi, df_ccsei_pivot, tabel_be_avg,
              df_eng_final_pivot, area_perbaikan]

list_nama_simpan = ['data mentah','crosstab rata', 'crosstab transpose', 'aspek','subkpi','kpi','dimensi','index akhir',
                   'engagement index aspek','engagement index final','aspek perbaikan']

writer = pd.ExcelWriter(nama_output)
for i in range(len(list_simpan)):
    if (list_nama_simpan[i]=='data mentah'):
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i], index=False, freeze_panes=(1, 3))
        worksheet = writer.sheets[list_nama_simpan[i]]
        worksheet.autofilter(0, 0, 0, 2) #first row, first col, last row, last col
    elif (list_nama_simpan[i]=='crosstab rata'):
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i], freeze_panes=(6,2))        
        worksheet = writer.sheets[list_nama_simpan[i]]
        worksheet.autofilter(5, 1, 5, 1)
    elif (list_nama_simpan[i]=='crosstab transpose'):
        list_simpan[i].set_index('KODE').to_excel(writer,sheet_name=list_nama_simpan[i], freeze_panes=(3,1))
    else:
        max_col = len(list_simpan[i].index.names)-1
        list_simpan[i].to_excel(writer,sheet_name=list_nama_simpan[i], freeze_panes=(1, max_col+1) , merge_cells=False)
        worksheet = writer.sheets[list_nama_simpan[i]]
        worksheet.autofilter(0, 0, 0, max_col)
writer.save() 